In [37]:
import csv

data = []

with open('./ml-latest-small/ratings.csv') as ratings:
    rating_reader = csv.reader(ratings, delimiter=',')
    line_count = 0
    for row in rating_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            # row is a list of string with the form [userId, movieId, rating, timestamp]
            data.append(row)
            line_count += 1
    print('Processed ' + str(line_count) + ' rows.')
genres=[]
with open('./ml-latest-small/movies.csv',errors='ignore') as ratings:
    rating_reader = csv.reader(ratings, delimiter=',')
    line_count = 0
    for row in rating_reader:
        if line_count == 0:
            line_count += 1
            continue
        else:
            # row is a list of string with the form [userId, movieId, rating, timestamp]
            genres.append(row)
            line_count += 1
    print('Processed ' + str(line_count) + ' rows.')    

Processed 100837 rows.
Processed 9743 rows.


In [38]:
from random import shuffle
from collections import defaultdict
shuffle(data)
for d in data: d[2]=float(d[2])
train = data[0:len(data)//3]
validation = data[len(data)//3:len(data)//3*2]
test = data[len(data)//3*2:len(data)]


userMovie = defaultdict(list)
#userid->[movieid,rating]
MovieUser = defaultdict(list)
#movieid->[userid,rating]

for d in train:
    u = d[0]
    i = d[1]
    rating = d[2]
    userMovie[u].append([i,rating])
    MovieUser[i].append([u,rating])

In [40]:

# userGenres=defaultdict(set)
movieGenres=defaultdict(set)
gen=set()
for d in genres:
    mid = d[0]
    gs = d[2].split("|")
    for g in gs:
        gen.add(g)
        movieGenres[mid].add(g)
gen=list(gen)


20


In [41]:
#rating by jaccard similarity
#if watched return the value, else use jaccard
def predict(movie,user):
    genrem=movieGenres[movie]
    denom=0.0
    num=0.0
    for m in userMovie[user]:
#         print(m[1])
        #if movie == m[0]: return float(m[1])
        sim = jaccard(genrem,movieGenres[m[0]])
        denom+=sim
        num+=sim*m[1]
    return num/denom    
def jaccard(a,b):
#     print(a,b)
    intersect=a&b
    union=a|b
    return len(intersect)/len(union)


predict('162350','596')

4.0

In [50]:
#rating by cosine similarity
#if watched return the value, else use jaccard
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
def predict(movie,user):
    genrem=movieGenres[movie]
    denom=0.0
    num=0.0
    for m in userMovie[user]:
#         print(m[1])
        #if movie == m[0]: return float(m[1])
        sim = cosine(genrem,movieGenres[m[0]])
        denom+=sim
        num+=sim*m[1]
    return num/denom

def cosine(a,b):
    list1=[0]*len(gen)
    list2=[0]*len(gen)
    for i in range(len(gen)):
        if gen[i] in a: list1[i]=1
        if gen[i] in b: list2[i]=1
    res=cosine_similarity(list1,list2)[0,0]
#     print(type(res))
    return res 
predict('318','596')

3.4465107243865107